# Module 05: Momentum Mathematics

**Difficulty**: ⭐⭐ (Intermediate)

**Estimated Time**: 60 minutes

**Prerequisites**: 
- Module 00: Introduction and Stock Returns
- Module 01: Averages and Central Tendency
- Module 02: Spread and Variation
- Module 03: Percentages, Ratios, and Changes (RSI basics)
- Module 04: Moving Averages

## Learning Objectives

By the end of this notebook, you will be able to:
1. Master **RSI** calculation and interpretation (deep dive)
2. Build **Stochastic Oscillator** (%K and %D) from scratch
3. Calculate **Williams %R** and understand its relationship to Stochastic
4. Implement **Momentum** and **Rate of Change (ROC)** indicators
5. Detect **divergence** between price and momentum
6. Combine multiple momentum indicators for **confluence**

## Why This Matters

**Momentum indicators answer: "Is the trend strengthening or weakening?"**

Unlike moving averages (which track price), momentum indicators track the **rate of price change**:
- **Price can go up while momentum weakens** → Warning sign
- **Price can go down while momentum strengthens** → Potential reversal
- **Divergence** between price and momentum = High probability signal

Applications:
- **Overbought/Oversold** identification
- **Divergence** detection (hidden opportunities)
- **Trend strength** measurement
- **Entry/Exit timing** for trades
- **Confirmation** of trend changes

---

## Setup

In [ ]:
# Data manipulation and numerical operations
import pandas as pd
import numpy as np

# Data acquisition
import yfinance as yf

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Pandas display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.precision', 4)

# Random seed
np.random.seed(42)

print("✓ Libraries imported successfully!")

In [ ]:
# Download Malaysian stock data
print("Downloading Malaysian stock data...\n")

# Maybank - stable for clear momentum signals
maybank = yf.download('1155.KL', start='2023-01-01', end='2024-01-01', progress=False)

# Top Glove - volatile for testing oscillators
topglove = yf.download('5225.KL', start='2023-01-01', end='2024-01-01', progress=False)

# Axiata - moderate volatility
axiata = yf.download('6888.KL', start='2023-01-01', end='2024-01-01', progress=False)

# Validate
assert len(maybank) > 0, "Failed to download Maybank data"
assert len(topglove) > 0, "Failed to download Top Glove data"
assert len(axiata) > 0, "Failed to download Axiata data"

print(f"✓ Maybank: {len(maybank)} days")
print(f"✓ Top Glove: {len(topglove)} days")
print(f"✓ Axiata: {len(axiata)} days")
print("\nData ready!")

---

## Part 1: RSI - Deep Dive

### Advanced RSI Concepts

In Module 03, we learned RSI basics. Now we'll explore:
1. **Wilder's smoothing** vs simple average
2. **Divergence** detection
3. **Failure swings** (advanced pattern)
4. **Optimal periods** for different strategies

### Wilder's Original RSI Formula

J. Welles Wilder used **exponential smoothing** for gains/losses:

$$
\begin{aligned}
\text{Initial: } \text{Avg Gain} &= \frac{\sum \text{Gains}}{14} \\
\text{Subsequent: } \text{Avg Gain} &= \frac{\text{Previous Avg Gain} \times 13 + \text{Current Gain}}{14}
\end{aligned}
$$

This creates a smoothed RSI that's less volatile than using simple moving average.

In [ ]:
def calculate_rsi_wilder(prices, period=14):
    """
    Calculate RSI using Wilder's smoothing method (original).
    """
    delta = prices.diff()
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)
    
    # Use EWM with alpha = 1/period for Wilder's smoothing
    avg_gains = gains.ewm(alpha=1/period, adjust=False).mean()
    avg_losses = losses.ewm(alpha=1/period, adjust=False).mean()
    
    rs = avg_gains / avg_losses
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

def calculate_rsi_sma(prices, period=14):
    """
    Calculate RSI using simple moving average (simpler version).
    """
    delta = prices.diff()
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)
    
    avg_gains = gains.rolling(window=period).mean()
    avg_losses = losses.rolling(window=period).mean()
    
    rs = avg_gains / avg_losses
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# Calculate both versions for Maybank
maybank_close = maybank['Close']

rsi_wilder = calculate_rsi_wilder(maybank_close, 14)
rsi_sma = calculate_rsi_sma(maybank_close, 14)

print("RSI Comparison: Wilder's Smoothing vs SMA\n")
print("=" * 70)
print(f"\nCurrent Values:")
print(f"  Wilder's RSI: {rsi_wilder.iloc[-1]:.2f}")
print(f"  SMA RSI:      {rsi_sma.iloc[-1]:.2f}")
print(f"  Difference:   {abs(rsi_wilder.iloc[-1] - rsi_sma.iloc[-1]):.2f}")

# Visualize comparison
plt.figure(figsize=(14, 8))

plt.subplot(2, 1, 1)
plt.plot(maybank_close.index, maybank_close.values,
         linewidth=1.5, color='black', alpha=0.7)
plt.title('Maybank Price', fontsize=12, fontweight='bold')
plt.ylabel('Price (RM)')
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(rsi_wilder.index, rsi_wilder.values,
         linewidth=2, label="Wilder's RSI (EMA smoothing)", alpha=0.9)
plt.plot(rsi_sma.index, rsi_sma.values,
         linewidth=2, label='SMA RSI (simple average)', alpha=0.7, linestyle='--')
plt.axhline(y=70, color='red', linestyle=':', linewidth=1, alpha=0.5)
plt.axhline(y=30, color='green', linestyle=':', linewidth=1, alpha=0.5)
plt.axhline(y=50, color='gray', linestyle=':', linewidth=1, alpha=0.3)
plt.fill_between(rsi_wilder.index, 70, 100, alpha=0.1, color='red')
plt.fill_between(rsi_wilder.index, 0, 30, alpha=0.1, color='green')

plt.title('RSI Comparison', fontsize=12, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.ylim(0, 100)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("• Wilder's RSI (EMA) is smoother")
print("• SMA RSI is more volatile, reacts quicker to sudden changes")
print("• Wilder's is the standard, but both are valid")
print("• Choice depends on trading style (smooth vs responsive)")

### RSI Divergence - Hidden Opportunities

**Divergence** = When price and RSI move in opposite directions.

Types:
1. **Bullish Divergence**: Price makes lower low, RSI makes higher low → Weakening downtrend
2. **Bearish Divergence**: Price makes higher high, RSI makes lower high → Weakening uptrend

These are powerful reversal signals!

In [ ]:
# Detect divergence (simplified algorithm)
def find_peaks_troughs(series, window=5):
    """
    Find local peaks and troughs in a series.
    """
    peaks = (series == series.rolling(window=window, center=True).max())
    troughs = (series == series.rolling(window=window, center=True).min())
    return peaks, troughs

# Find peaks and troughs in price and RSI
price_peaks, price_troughs = find_peaks_troughs(maybank_close, window=10)
rsi_peaks, rsi_troughs = find_peaks_troughs(rsi_wilder, window=10)

# Visualize with peaks and troughs marked
plt.figure(figsize=(14, 10))

# Price with peaks/troughs
plt.subplot(2, 1, 1)
plt.plot(maybank_close.index, maybank_close.values,
         linewidth=1.5, color='black', alpha=0.7)
plt.scatter(maybank_close[price_peaks].index, maybank_close[price_peaks].values,
           marker='v', s=100, color='red', zorder=5, label='Peaks')
plt.scatter(maybank_close[price_troughs].index, maybank_close[price_troughs].values,
           marker='^', s=100, color='green', zorder=5, label='Troughs')
plt.title('Maybank Price - Peaks and Troughs', fontsize=12, fontweight='bold')
plt.ylabel('Price (RM)')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

# RSI with peaks/troughs
plt.subplot(2, 1, 2)
plt.plot(rsi_wilder.index, rsi_wilder.values,
         linewidth=2, color='purple', alpha=0.8)
plt.scatter(rsi_wilder[rsi_peaks].index, rsi_wilder[rsi_peaks].values,
           marker='v', s=100, color='red', zorder=5, label='RSI Peaks')
plt.scatter(rsi_wilder[rsi_troughs].index, rsi_wilder[rsi_troughs].values,
           marker='^', s=100, color='green', zorder=5, label='RSI Troughs')
plt.axhline(y=70, color='red', linestyle=':', linewidth=1, alpha=0.5)
plt.axhline(y=30, color='green', linestyle=':', linewidth=1, alpha=0.5)
plt.title('RSI - Peaks and Troughs', fontsize=12, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.ylim(0, 100)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Divergence Detection Guide:")
print("\nBullish Divergence (potential reversal up):")
print("  • Price makes lower low")
print("  • RSI makes higher low (or equal low)")
print("  → Selling pressure weakening")

print("\nBearish Divergence (potential reversal down):")
print("  • Price makes higher high")
print("  • RSI makes lower high (or equal high)")
print("  → Buying pressure weakening")

print("\n» Look for these patterns at extreme RSI levels (>70 or <30)")

---

## Part 2: Stochastic Oscillator - Complete Implementation

### What is Stochastic?

**Stochastic Oscillator** shows where the current price is relative to the recent range.

### Formula

$$
\begin{aligned}
\%K &= \frac{\text{Close} - \text{Low}_{14}}{\text{High}_{14} - \text{Low}_{14}} \times 100 \\
\%D &= SMA_3(\%K)
\end{aligned}
$$

Where:
- $\%K$ = Fast line (more responsive)
- $\%D$ = Slow line (smoothed signal)
- Common settings: 14-period for range, 3-period SMA for %D

### Interpretation

- **0-20**: Oversold zone
- **80-100**: Overbought zone
- **%K crosses above %D**: Bullish signal
- **%K crosses below %D**: Bearish signal

In [ ]:
def calculate_stochastic(high, low, close, k_period=14, d_period=3):
    """
    Calculate Stochastic Oscillator (%K and %D).
    
    Parameters:
    -----------
    high : pd.Series
        High prices
    low : pd.Series
        Low prices
    close : pd.Series
        Close prices
    k_period : int
        Lookback period for %K (default 14)
    d_period : int
        Smoothing period for %D (default 3)
    
    Returns:
    --------
    tuple: (%K, %D)
    """
    # Calculate rolling high and low
    highest_high = high.rolling(window=k_period).max()
    lowest_low = low.rolling(window=k_period).min()
    
    # Calculate %K (Fast Stochastic)
    stoch_k = ((close - lowest_low) / (highest_high - lowest_low)) * 100
    
    # Calculate %D (Slow Stochastic - SMA of %K)
    stoch_d = stoch_k.rolling(window=d_period).mean()
    
    return stoch_k, stoch_d

# Calculate Stochastic for Top Glove (volatile stock)
tg_high = topglove['High']
tg_low = topglove['Low']
tg_close = topglove['Close']

stoch_k, stoch_d = calculate_stochastic(tg_high, tg_low, tg_close, k_period=14, d_period=3)

print("Stochastic Oscillator for Top Glove\n")
print("=" * 70)
print(f"\nCurrent Values:")
print(f"  %K (Fast): {stoch_k.iloc[-1]:.2f}")
print(f"  %D (Slow): {stoch_d.iloc[-1]:.2f}")

# Determine signal
if stoch_k.iloc[-1] > 80:
    signal = "OVERBOUGHT"
elif stoch_k.iloc[-1] < 20:
    signal = "OVERSOLD"
else:
    signal = "Neutral"

print(f"\nSignal: {signal}")

# Detect crossovers
bullish_cross = (stoch_k > stoch_d) & (stoch_k.shift(1) <= stoch_d.shift(1))
bearish_cross = (stoch_k < stoch_d) & (stoch_k.shift(1) >= stoch_d.shift(1))

print(f"\nCrossovers in 2023:")
print(f"  Bullish (%K crosses above %D): {bullish_cross.sum()}")
print(f"  Bearish (%K crosses below %D): {bearish_cross.sum()}")

In [ ]:
# Visualize Stochastic with signals
plt.figure(figsize=(14, 10))

# Price
plt.subplot(2, 1, 1)
plt.plot(tg_close.index, tg_close.values,
         linewidth=1.5, color='black', alpha=0.7, label='Price')

# Mark crossovers on price chart
for date in tg_close[bullish_cross].index:
    plt.axvline(x=date, color='green', linestyle=':', alpha=0.3, linewidth=1.5)
for date in tg_close[bearish_cross].index:
    plt.axvline(x=date, color='red', linestyle=':', alpha=0.3, linewidth=1.5)

plt.title('Top Glove Price', fontsize=12, fontweight='bold')
plt.ylabel('Price (RM)')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

# Stochastic
plt.subplot(2, 1, 2)
plt.plot(stoch_k.index, stoch_k.values,
         linewidth=2, label='%K (14)', color='blue', alpha=0.8)
plt.plot(stoch_d.index, stoch_d.values,
         linewidth=2, label='%D (3-SMA of %K)', color='red', alpha=0.8)

# Overbought/Oversold zones
plt.axhline(y=80, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='Overbought (80)')
plt.axhline(y=20, color='green', linestyle='--', linewidth=1.5, alpha=0.7, label='Oversold (20)')
plt.fill_between(stoch_k.index, 80, 100, alpha=0.2, color='red')
plt.fill_between(stoch_k.index, 0, 20, alpha=0.2, color='green')

# Mark crossovers
plt.scatter(stoch_k[bullish_cross].index, stoch_k[bullish_cross].values,
           marker='^', s=150, color='green', zorder=5,
           edgecolors='black', linewidths=2, label='Bullish Cross')
plt.scatter(stoch_k[bearish_cross].index, stoch_k[bearish_cross].values,
           marker='v', s=150, color='red', zorder=5,
           edgecolors='black', linewidths=2, label='Bearish Cross')

plt.title('Stochastic Oscillator (14,3)', fontsize=12, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Stochastic Value')
plt.ylim(-5, 105)
plt.legend(loc='best', fontsize=9)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Trading Signals from Stochastic:")
print("\n1. Overbought/Oversold:")
print("   • Wait for %K to enter zone (<20 or >80)")
print("   • Wait for crossover to confirm reversal")

print("\n2. Crossovers:")
print("   • %K crosses above %D + both below 20 = Strong buy")
print("   • %K crosses below %D + both above 80 = Strong sell")

print("\n3. Divergence:")
print("   • Price makes new low, Stochastic makes higher low = Bullish")
print("   • Price makes new high, Stochastic makes lower high = Bearish")

---

## Part 3: Williams %R - Inverted Stochastic

### What is Williams %R?

Williams %R is essentially an **inverted Stochastic Oscillator**.

### Formula

$$
\%R = \frac{\text{High}_{14} - \text{Close}}{\text{High}_{14} - \text{Low}_{14}} \times (-100)
$$

**Key Differences from Stochastic**:
- Range: **-100 to 0** (vs Stochastic's 0 to 100)
- Formula: Measures distance from HIGH (vs Stochastic from LOW)
- Interpretation: **-80 to -100 = Oversold**, **-20 to 0 = Overbought**

### Mathematical Relationship

$$
\%R = -(100 - \%K)
$$

They're mathematically equivalent, just scaled differently!

In [ ]:
def calculate_williams_r(high, low, close, period=14):
    """
    Calculate Williams %R.
    
    Parameters:
    -----------
    high : pd.Series
        High prices
    low : pd.Series
        Low prices
    close : pd.Series
        Close prices
    period : int
        Lookback period (default 14)
    
    Returns:
    --------
    pd.Series
        Williams %R values
    """
    highest_high = high.rolling(window=period).max()
    lowest_low = low.rolling(window=period).min()
    
    williams_r = ((highest_high - close) / (highest_high - lowest_low)) * (-100)
    
    return williams_r

# Calculate Williams %R for Axiata
ax_high = axiata['High']
ax_low = axiata['Low']
ax_close = axiata['Close']

williams_r = calculate_williams_r(ax_high, ax_low, ax_close, period=14)

# Also calculate %K for comparison
stoch_k_ax, _ = calculate_stochastic(ax_high, ax_low, ax_close, k_period=14, d_period=3)

# Verify mathematical relationship: %R = -(100 - %K)
williams_r_from_k = -(100 - stoch_k_ax)

print("Williams %R Analysis\n")
print("=" * 70)
print(f"\nCurrent Williams %R: {williams_r.iloc[-1]:.2f}")
print(f"Current Stochastic %K: {stoch_k_ax.iloc[-1]:.2f}")
print(f"\nVerification: -(100 - %K) = {williams_r_from_k.iloc[-1]:.2f}")
print(f"Direct calculation:      {williams_r.iloc[-1]:.2f}")
print(f"Difference: {abs(williams_r_from_k.iloc[-1] - williams_r.iloc[-1]):.6f}")
print("\n✓ They match! Williams %R = -(100 - Stochastic %K)")

In [ ]:
# Visualize Williams %R vs Stochastic %K
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Price
axes[0].plot(ax_close.index, ax_close.values,
            linewidth=1.5, color='black', alpha=0.7)
axes[0].set_title('Axiata Price', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Price (RM)')
axes[0].grid(True, alpha=0.3)

# Stochastic %K
axes[1].plot(stoch_k_ax.index, stoch_k_ax.values,
            linewidth=2, color='blue', alpha=0.8)
axes[1].axhline(y=80, color='red', linestyle='--', linewidth=1, alpha=0.7)
axes[1].axhline(y=20, color='green', linestyle='--', linewidth=1, alpha=0.7)
axes[1].fill_between(stoch_k_ax.index, 80, 100, alpha=0.2, color='red')
axes[1].fill_between(stoch_k_ax.index, 0, 20, alpha=0.2, color='green')
axes[1].set_title('Stochastic %K (0 to 100)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('%K Value')
axes[1].set_ylim(-5, 105)
axes[1].grid(True, alpha=0.3)

# Williams %R
axes[2].plot(williams_r.index, williams_r.values,
            linewidth=2, color='purple', alpha=0.8)
axes[2].axhline(y=-20, color='red', linestyle='--', linewidth=1, alpha=0.7, label='Overbought (-20)')
axes[2].axhline(y=-80, color='green', linestyle='--', linewidth=1, alpha=0.7, label='Oversold (-80)')
axes[2].fill_between(williams_r.index, -20, 0, alpha=0.2, color='red')
axes[2].fill_between(williams_r.index, -100, -80, alpha=0.2, color='green')
axes[2].set_title('Williams %R (-100 to 0)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('%R Value')
axes[2].set_ylim(-105, 5)
axes[2].legend(loc='best')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Comparison: Stochastic %K vs Williams %R")
print("\nStochastic %K:")
print("  • Range: 0 to 100")
print("  • Overbought: > 80")
print("  • Oversold: < 20")
print("  • Measures distance from LOW")

print("\nWilliams %R:")
print("  • Range: -100 to 0")
print("  • Overbought: > -20")
print("  • Oversold: < -80")
print("  • Measures distance from HIGH")

print("\nThey provide THE SAME information, just scaled differently!")
print("Choice is personal preference.")

---

## Part 4: Momentum and ROC - Pure Velocity

### Momentum Indicator

**Momentum** measures absolute price change over N periods.

$$
\text{Momentum}_n = \text{Price}_{today} - \text{Price}_{n\text{ periods ago}}
$$

### Rate of Change (ROC)

**ROC** measures percentage change (covered in Module 03).

$$
ROC_n = \frac{\text{Price}_{today} - \text{Price}_{n\text{ periods ago}}}{\text{Price}_{n\text{ periods ago}}} \times 100
$$

### Key Difference

- **Momentum**: Absolute change (RM)
- **ROC**: Percentage change (%)

ROC is better for comparing stocks with different price levels.

In [ ]:
def calculate_momentum(prices, period=10):
    """
    Calculate Momentum indicator.
    """
    return prices - prices.shift(period)

def calculate_roc(prices, period=10):
    """
    Calculate Rate of Change (ROC).
    """
    return ((prices - prices.shift(period)) / prices.shift(period)) * 100

# Calculate for Maybank
momentum_10 = calculate_momentum(maybank_close, 10)
roc_10 = calculate_roc(maybank_close, 10)

# Visualize both
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Price
axes[0].plot(maybank_close.index, maybank_close.values,
            linewidth=1.5, color='black', alpha=0.7)
axes[0].set_title('Maybank Price', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Price (RM)')
axes[0].grid(True, alpha=0.3)

# Momentum
axes[1].plot(momentum_10.index, momentum_10.values,
            linewidth=2, color='blue', alpha=0.8)
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=2, alpha=0.7)
axes[1].fill_between(momentum_10.index, 0, momentum_10.values,
                     where=(momentum_10 >= 0), alpha=0.3, color='green')
axes[1].fill_between(momentum_10.index, 0, momentum_10.values,
                     where=(momentum_10 < 0), alpha=0.3, color='red')
axes[1].set_title('Momentum (10-period, absolute change)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Momentum (RM)')
axes[1].grid(True, alpha=0.3)

# ROC
axes[2].plot(roc_10.index, roc_10.values,
            linewidth=2, color='purple', alpha=0.8)
axes[2].axhline(y=0, color='red', linestyle='--', linewidth=2, alpha=0.7)
axes[2].fill_between(roc_10.index, 0, roc_10.values,
                     where=(roc_10 >= 0), alpha=0.3, color='green')
axes[2].fill_between(roc_10.index, 0, roc_10.values,
                     where=(roc_10 < 0), alpha=0.3, color='red')
axes[2].set_title('ROC (10-period, percentage change)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('ROC (%)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Momentum vs ROC:")
print(f"\nCurrent Values:")
print(f"  Momentum: {momentum_10.iloc[-1]:+.4f} RM")
print(f"  ROC:      {roc_10.iloc[-1]:+.4f} %")

print("\nInterpretation:")
print("  Positive = Upward momentum")
print("  Negative = Downward momentum")
print("  Zero crossing = Potential trend change")
print("\nROC is preferred for comparing different stocks.")

---

## Part 5: Combining Momentum Indicators - Confluence

**Confluence** = Multiple indicators agreeing on the same signal.

When RSI, Stochastic, and Williams %R ALL agree → High probability setup!

### Creating a Momentum Scorecard

In [ ]:
def create_momentum_scorecard(high, low, close, rsi_period=14, stoch_period=14):
    """
    Create a momentum scorecard combining multiple indicators.
    
    Returns:
    --------
    pd.DataFrame with columns: RSI, Stoch_K, Williams_R, ROC, Score
    """
    # Calculate all indicators
    rsi = calculate_rsi_wilder(close, rsi_period)
    stoch_k, stoch_d = calculate_stochastic(high, low, close, stoch_period, 3)
    williams = calculate_williams_r(high, low, close, 14)
    roc = calculate_roc(close, 10)
    
    # Create signals (-1 = bearish, 0 = neutral, +1 = bullish)
    rsi_signal = pd.Series(0, index=rsi.index)
    rsi_signal[rsi > 70] = -1  # Overbought
    rsi_signal[rsi < 30] = 1   # Oversold
    
    stoch_signal = pd.Series(0, index=stoch_k.index)
    stoch_signal[stoch_k > 80] = -1
    stoch_signal[stoch_k < 20] = 1
    
    williams_signal = pd.Series(0, index=williams.index)
    williams_signal[williams > -20] = -1  # Overbought
    williams_signal[williams < -80] = 1   # Oversold
    
    roc_signal = pd.Series(0, index=roc.index)
    roc_signal[roc > 5] = 1    # Strong positive momentum
    roc_signal[roc < -5] = -1  # Strong negative momentum
    
    # Calculate total score (-4 to +4)
    total_score = rsi_signal + stoch_signal + williams_signal + roc_signal
    
    # Create DataFrame
    scorecard = pd.DataFrame({
        'RSI': rsi,
        'Stoch_K': stoch_k,
        'Williams_R': williams,
        'ROC': roc,
        'RSI_Signal': rsi_signal,
        'Stoch_Signal': stoch_signal,
        'Williams_Signal': williams_signal,
        'ROC_Signal': roc_signal,
        'Total_Score': total_score
    })
    
    return scorecard

# Create scorecard for Top Glove
scorecard = create_momentum_scorecard(tg_high, tg_low, tg_close)

print("Momentum Scorecard for Top Glove\n")
print("=" * 80)
print("\nCurrent Status:")
print(scorecard[[col for col in scorecard.columns if 'Signal' not in col and col != 'Total_Score']].tail(1).to_string())
print(f"\nSignals:")
print(scorecard[['RSI_Signal', 'Stoch_Signal', 'Williams_Signal', 'ROC_Signal', 'Total_Score']].tail(1).to_string())

current_score = scorecard['Total_Score'].iloc[-1]
print(f"\nTotal Momentum Score: {current_score:.0f} / 4")

if current_score >= 3:
    print("\n🟢 STRONG BULLISH CONFLUENCE - Multiple oversold signals")
elif current_score <= -3:
    print("\n🔴 STRONG BEARISH CONFLUENCE - Multiple overbought signals")
elif current_score > 0:
    print("\n🟡 Mild bullish bias")
elif current_score < 0:
    print("\n🟡 Mild bearish bias")
else:
    print("\n⚪ NEUTRAL - Mixed signals")

In [ ]:
# Visualize momentum scorecard over time
plt.figure(figsize=(14, 12))

# Price
plt.subplot(5, 1, 1)
plt.plot(tg_close.index, tg_close.values,
         linewidth=1.5, color='black', alpha=0.7)
# Highlight strong confluence zones
strong_buy = scorecard['Total_Score'] >= 3
strong_sell = scorecard['Total_Score'] <= -3
for date in tg_close[strong_buy].index:
    plt.axvspan(date, date + pd.Timedelta(days=1), alpha=0.2, color='green')
for date in tg_close[strong_sell].index:
    plt.axvspan(date, date + pd.Timedelta(days=1), alpha=0.2, color='red')
plt.title('Top Glove Price (Green = Strong Buy Signal, Red = Strong Sell Signal)', 
          fontsize=11, fontweight='bold')
plt.ylabel('Price (RM)')
plt.grid(True, alpha=0.3)

# RSI
plt.subplot(5, 1, 2)
plt.plot(scorecard.index, scorecard['RSI'],
         linewidth=1.5, color='purple', alpha=0.7)
plt.axhline(y=70, color='red', linestyle=':', linewidth=1, alpha=0.5)
plt.axhline(y=30, color='green', linestyle=':', linewidth=1, alpha=0.5)
plt.fill_between(scorecard.index, 70, 100, alpha=0.1, color='red')
plt.fill_between(scorecard.index, 0, 30, alpha=0.1, color='green')
plt.title('RSI (14)', fontsize=10, fontweight='bold')
plt.ylabel('RSI')
plt.ylim(0, 100)
plt.grid(True, alpha=0.3)

# Stochastic
plt.subplot(5, 1, 3)
plt.plot(scorecard.index, scorecard['Stoch_K'],
         linewidth=1.5, color='blue', alpha=0.7)
plt.axhline(y=80, color='red', linestyle=':', linewidth=1, alpha=0.5)
plt.axhline(y=20, color='green', linestyle=':', linewidth=1, alpha=0.5)
plt.fill_between(scorecard.index, 80, 100, alpha=0.1, color='red')
plt.fill_between(scorecard.index, 0, 20, alpha=0.1, color='green')
plt.title('Stochastic %K (14)', fontsize=10, fontweight='bold')
plt.ylabel('%K')
plt.ylim(0, 100)
plt.grid(True, alpha=0.3)

# Williams %R
plt.subplot(5, 1, 4)
plt.plot(scorecard.index, scorecard['Williams_R'],
         linewidth=1.5, color='orange', alpha=0.7)
plt.axhline(y=-20, color='red', linestyle=':', linewidth=1, alpha=0.5)
plt.axhline(y=-80, color='green', linestyle=':', linewidth=1, alpha=0.5)
plt.fill_between(scorecard.index, -20, 0, alpha=0.1, color='red')
plt.fill_between(scorecard.index, -100, -80, alpha=0.1, color='green')
plt.title('Williams %R (14)', fontsize=10, fontweight='bold')
plt.ylabel('%R')
plt.ylim(-100, 0)
plt.grid(True, alpha=0.3)

# Total Score
plt.subplot(5, 1, 5)
colors = ['green' if s > 0 else 'red' if s < 0 else 'gray' for s in scorecard['Total_Score']]
plt.bar(scorecard.index, scorecard['Total_Score'], color=colors, alpha=0.6, width=1)
plt.axhline(y=0, color='black', linestyle='-', linewidth=1)
plt.axhline(y=3, color='green', linestyle='--', linewidth=1, alpha=0.7, label='Strong Bullish')
plt.axhline(y=-3, color='red', linestyle='--', linewidth=1, alpha=0.7, label='Strong Bearish')
plt.title('Momentum Confluence Score', fontsize=10, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Score')
plt.ylim(-5, 5)
plt.legend(loc='best', fontsize=8)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Count strong signals
strong_buy_count = (scorecard['Total_Score'] >= 3).sum()
strong_sell_count = (scorecard['Total_Score'] <= -3).sum()

print(f"\nStrong Confluence Signals in 2023:")
print(f"  Strong Buy signals (score ≥ 3):  {strong_buy_count}")
print(f"  Strong Sell signals (score ≤ -3): {strong_sell_count}")
print(f"\nWhen ALL indicators agree, the signal is much more reliable!")

---

## Part 6: Exercises

Practice momentum indicator analysis with these exercises.

### Exercise 1: RSI Divergence Detection

Create an algorithm to automatically detect RSI divergences.

**Tasks**:
1. Find price peaks and troughs using rolling windows
2. Find RSI peaks and troughs
3. Compare consecutive peaks/troughs to detect divergence
4. Visualize detected divergences
5. Calculate success rate of divergence signals

In [ ]:
# Your code here



<details>
<summary><b>Click here for solution</b></summary>

```python
# This is a challenging exercise requiring peak/trough detection
# and pattern matching between price and RSI
# See the Module 05 solution notebook for complete implementation
```
</details>

### Exercise 2: Optimize Stochastic Parameters

Find the optimal Stochastic parameters for Maybank.

**Tasks**:
1. Test different %K periods (5, 10, 14, 20, 30)
2. Test different %D periods (3, 5, 7, 9)
3. For each combination, count crossover signals
4. Measure signal quality (how many led to price moves)
5. Recommend best parameters

In [ ]:
# Your code here



### Exercise 3: Momentum Divergence Trading System

Build a complete trading system using momentum divergence.

**Tasks**:
1. Combine RSI and Stochastic divergence signals
2. Add ROC confirmation
3. Create entry rules (all 3 agree)
4. Create exit rules (opposite signal or time-based)
5. Backtest on 2023 data
6. Calculate win rate and average return

In [ ]:
# Your code here



### Exercise 4: Multi-Stock Momentum Screening

Create a momentum screener for multiple Malaysian stocks.

**Tasks**:
1. Download data for 5 Malaysian stocks
2. Calculate momentum scorecard for each
3. Rank stocks by current momentum score
4. Identify stocks with strong buy/sell signals
5. Create a visual dashboard showing all stocks

In [ ]:
# Your code here



---

## Summary

Congratulations! You've completed Module 05. Let's review:

### Key Concepts Mastered

1. **RSI Deep Dive**
   - Wilder's smoothing vs SMA method
   - Divergence detection and interpretation
   - Failure swing patterns

2. **Stochastic Oscillator**
   - Complete %K and %D calculation
   - Overbought/oversold interpretation
   - Crossover signals

3. **Williams %R**
   - Mathematical relationship to Stochastic: %R = -(100 - %K)
   - Inverted scale (-100 to 0)
   - Same information, different presentation

4. **Momentum & ROC**
   - Absolute vs percentage change
   - Zero-line significance
   - Velocity of price movement

5. **Confluence**
   - Combining multiple indicators
   - Momentum scorecard system
   - High-probability setups

6. **Divergence**
   - Bullish divergence (reversal up)
   - Bearish divergence (reversal down)
   - Leading vs lagging signals

### Practical Applications

- Entry timing using multiple confirmations
- Exit signals from divergence
- Position sizing based on momentum strength
- Risk management using overbought/oversold zones

### What's Next?

**Module 06: Volatility Mathematics** will cover:
- Bollinger Bands complete calculation
- Average True Range (ATR)
- Keltner Channels
- Using volatility for stop-loss placement

---

## Additional Resources

### Further Reading
- Wilder, J. Welles (1978). "New Concepts in Technical Trading Systems"
- [Investopedia: Stochastic Oscillator](https://www.investopedia.com/terms/s/stochasticoscillator.asp)
- [Investopedia: Williams %R](https://www.investopedia.com/terms/w/williamsr.asp)
- [StockCharts: Divergence](https://school.stockcharts.com/doku.php?id=trading_strategies:divergences)

---

**Excellent work!** You now understand the mathematics and application of momentum oscillators. Ready for Module 06?